In [1]:
import numpy as np

This notebook computes the equilibrium grain size based on the models shown in Figure 4 of Mulyukova and Bercovici, 2018. The output can be compared to the ASPECT model output generated using the `grain_size_plunge.prm` input file.
The first step of reproducing these results is to convert all parameters into SI units.

We can compute the grain growth prefactor $k_g$ using the properties given in Table 1 in the paper, which gives us

\begin{equation}
k_g = G_0 (10^{-6})^q \frac {q}{p} \frac{1}{250}
\end{equation}

We want the growth rate $k_g$ to have units of m/s.

$G_0$ is qiven as $2 \cdot 10^4 \mu \text{m}^p/\text{s}$, so to get it in $\text{m}^p/\text{s}$, we have to divide by $(10^6)^p$.

In [2]:
p   = 2.
G_0 = 2e4 * (10**(-6))**p
q   = 4.
k_g = G_0 * q/p * 1./250. * (10**(-6))**(q-p)

print(k_g)

1.6000000000000002e-22


In [3]:
# define other constants from the paper here
q                       = 4.
grain_growth_prefactor  = k_g
grain_growth_activation = 3e5
f1                      = 5e-3
phase_distribution      = 0.6*0.4

hg                      = (2/np.pi)*(2/np.pi)
temperature             = 1100.
R_times_T               = 8.314*temperature

The setup we compare against uses a constant shear stress of 50 MPa (in ASPECT, we use a setup with simple shear for this).

In [4]:
# In our test case of simple shear, we apply vx = y/10^5 per year and vy = 0. Therefore, we only have the yx
# component of the strain rate tensor = 1/2*(vx/y) or 1/2*(1/10^5) per year.

# Computation of the prefactors:
# From Table 1
n = 3.
m = 3.

# A is given in MPa^(-n)/s, so we have to convert it to Pa^(-n)/s
A0 = 1.1e5 * (1.e-6)**n

# B0 is given in micro m^m/MPa/s, so we have to convert it to m^m/Pa/s
# In addition, diffusion creep depends on the grain size, so we have to include the conversion from grain size to roughness
B0 = 13.6 * (1.e-6)**m * 1.e-6 * (np.pi/2)**(-m)

strain_rate            = (1.e-5/(3600.*24.*365.25))/2.
stress                 = 50.e6

dislocation_activation = 530000
diffusion_activation   = 300000
stress_exponent        = 3
grain_size_exponent    = 3

# Note that there is no factor of 1/n in the exponent here
# because we compute the viscosity based on the stress, and not on the strain rate
A                      = A0 * np.exp(-dislocation_activation / R_times_T)
B                      = B0 * np.exp(-diffusion_activation / R_times_T)

# This is the field boundary roughness
r_f = (B / (A * stress**(n-1.))) ** (1./m)

# Convert back to grain size from roughness. This is now no longer what their equation says,
# but it is what they plot.
R_f = r_f * np.pi/2

print ("Field boundary grain size:", R_f, "m")

Field boundary grain size: 1.604955569793287e-05 m


This is the same field boundary grain size that is plotted in Figure 4 of Mulyukova & Bercovici (1.6e-5 m). Note that we still had to convert from roughness to grain size after calculating $r_f$ from their equation (8), so their naming scheme is a bit misleading (one would expect it to be field boundary roughness, not grain size).

In steady state, we can compute the equilibrium grain size in the pinned state using Eq. 8 of Mulyukova and Bercovici (2018):

\begin{equation}
0 = \frac {3 \phi_1 \phi_2 k_g \sqrt h_g^{-q}}{q R^{q-1}} \exp \left(-\frac{E_g}{R_gT}\right)  - \frac{f_{I} \sqrt h_g \Psi}{3 \phi_1 \phi_2} R^2
\end{equation}

Here, $\phi_1 \phi_2$ is the phase distribution function and $\sqrt{h_g} = 2/\pi$.

We can rewrite this equation as

\begin{equation}
0 = \frac{q f_{I} (\sqrt h_g R)^{q+1} \Psi}{(3 \phi_1 \phi_2)^2 k_g \exp \left(-\frac{E_g}{R_gT}\right)} - 1
\end{equation}

The shear heating term $\Psi$ is $2\dot\varepsilon\tau$ and the strain rate depends on the grain size. We can use equation (9) to rewrite the shear heating as

\begin{equation}
\Psi = 2 A  \tau^{n+1} \left( 1 + \left( \frac{r_f}{r} \right)^m \right)
\end{equation}

with the roughness $r = R \sqrt{h_g}$,
so

\begin{equation}
\Psi = 2 A  \tau^{n+1} \left( 1 + \left( \frac{r_f}{R \sqrt{h_g}} \right)^m \right)
\end{equation}

We can now see that we can not solve the equation analytically any more and instead need to find an iterative solution for the grain size. We implement this using a Newton-Raphson scheme:


In [5]:
# Compute the steady state grain size from eq (12) using N-R scheme.

G1 = grain_growth_prefactor * np.exp (-grain_growth_activation/R_times_T)

def steady_state_grain_size (grain_size):
  # we have to replace the roughness by grain size times 2/pi
  shear_heating = 2 * A * stress**(n + 1) * (1 + (r_f / (grain_size * np.sqrt(hg))) ** m)
  prefactors    = q * f1 * (grain_size * np.sqrt(hg))**(q + 1) / (phase_distribution * phase_distribution * 3 * 3 * G1)
  return (shear_heating * prefactors - 1)

def derivative (f, x, h):
    return (f(x + h) - f(x)) / h

def solve_r (f, x0, h):
    lastX = x0
    nextX = lastX + 10 * h  # different than lastX so loop starts OK

    while (abs(lastX - nextX) > h):
        newY  = steady_state_grain_size(nextX)
        print ("residual:", newY)
        lastX = nextX
        nextX = lastX - newY / derivative(f, lastX, h)  # update estimate
    return nextX

After having defined a function that can compute the equilibrium grain size, we can now call this function, starting from the initial grain size `x0`.

In [6]:
x0             = 1e-4;
h              = 1e-15;
grain_size     = solve_r (steady_state_grain_size, x0, h)
grain_size

residual: 4164445.1831699545
residual: 1365728.109414279
residual: 448244.85460342816
residual: 147343.03759121656
residual: 48574.41737952688
residual: 16101.159423364772
residual: 5389.150453665025
residual: 1832.1009018650964
residual: 634.4511041788161
residual: 219.46288967803093
residual: 70.1702735513497
residual: 19.164404733383968
residual: 4.649966270305967
residual: 0.9603983794093769
residual: 0.11775780149618598
residual: 0.003103649263562991
residual: 2.402064276596505e-06
residual: 1.2703171847761041e-12


7.63658755912162e-07

Our grain size (7.64e-5 m) is similar, but slightly smaller than plotted in Figure 4 of the paper.

In addition, we can also compare the viscosity (plotted in Figure 4, right panel):

In ASPECT, we compute the viscosity in the following way:

\begin{equation}
\eta = \frac{1}{A^{1/n}} R^m \dot\varepsilon_{II}^\frac{1-n}{n} \exp\left({\frac{E_d}{nR_gT}}\right)
\end{equation}

Note that there is a difference compared to the definition in the Mulyukova paper, which uses

\begin{equation}
\eta_\text{disl} = \frac{1}{2} \frac{1}{A \tau^{n-1}}  = \frac{1}{2} \frac{1}{A_0} \tau^{1-n} \exp\left({\frac{E_\text{disl}}{R_gT}}\right)\\
\eta_\text{diff} = \frac{1}{2} \frac{r^m}{B}  = \frac{1}{2} \frac{1}{B_0} R^m \sqrt{h_G}^{m} \exp\left({\frac{E_\text{diff}}{R_gT}}\right)
\end{equation}

They use the roughness rather than the grain size in their equation.
So we have to either scale our prefactor B for diffusion creep by $\sqrt{h_G}^m$, or convert from grain size to roughness first.

In [7]:
dislocation_viscosity = 1/(2 * A * stress ** (stress_exponent - 1))
diffusion_viscosity   = (grain_size*2/np.pi) ** (grain_size_exponent)/(2 * B)

viscosity             = diffusion_viscosity * dislocation_viscosity / (diffusion_viscosity + dislocation_viscosity)

print(viscosity)

2.8867405314847165e+18


Again, our viscosity (2.89e18 Pa s) is similar but slightly smaller than what is plotted in the figure (approximately 3.4e18 Pa s). This is expected, since we are in the diffusion creep regime, where a lower grain size reduces the viscosity.

Mulyukova and Bercovici give their equilibrium roughness in equation (12).
As a second check, we here convert our grain size to roughness and use that equation to check if our grain size fulfils it. In addition, we also use the grain size and $r_f$ from their Figure 4 (estimated) and check if they fulfil the equation.


In [8]:
# Equation (12) from Mulyukova & Bercovici
# Result should be 1
import numpy as np

# Use r and r_f from Figure 4 (so these are approximate)
# If I take Figure 4 as grain sizes and then convert them to roughness multiplying by sqrt(h_G)
r       = 8.04e-7 * (2. / np.pi)
r_f     = 1.6e-5 * (2. / np.pi)

# This is our analytical solution for r_f and r, again converted to roughness
r2       = 7.636587559121621e-07 / (np.pi / 2.)
r_f2     = 1.604955569793287e-05 / (np.pi / 2.)

# Parameters from Table 1
q       = 4.
f_I     = 5.e-3
eta     = 3. * 0.6 * 0.4
gamma   = 1.
tau     = 50.e6

# Compute A: A is given in MPa^(-n)/s, so we have to convert it to Pa^(-n)/s
n       = 3.
m       = 3.
A0      = 1.1e5 * (1.e-6)**n
E_disl  = 530000.
R       = 8.314
T       = 1100.

A       = A0 * np.exp(-E_disl / (R * T))

# Compute G_I
p      = 2.
G_0    = 2.e4 * (10.**(-6))**p
E_G    = 300000.
G_I    = G_0 * q/p * 1./250. * (10**(-6))**(q-p) * np.exp(- E_G / (R * T))

print("From Fig. 4:", 2. * q * f_I * A / (gamma * eta * eta * G_I) * r**(q+1) * tau**(n+1) * (1 + (r_f / r)**m))
print("From our notebook:", 2. * q * f_I * A / (gamma * eta * eta * G_I) * r2**(q+1) * tau**(n+1) * (1 + (r_f2 / r2)**m))

From Fig. 4: 1.0982283538272577
From our notebook: 1.0000000000000002


We can see that the results given in the paper only approximately fulfil the equation.

Our computed analytical solution gives us the correct result (which is expected assuming we have not made any mistakes in the implementation).